# Question Answering Model 

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

### import packages

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DefaultDataCollator,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer
)
import torch

import evaluate
import collections
from tqdm.auto import tqdm
import numpy as np

import warnings
import sys
import os
import re
import datetime

/mount/studenten-temp1/users/linku/AL_QA/.venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set cache directory if desired.
If you do not set a cache directory, then default values are used (usually '~/.cache')

In [3]:
CACHE_DIR=os.path.abspath(os.path.expanduser('cache'))
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/studenten-temp1/users/linku/AL_QA/cache
env: HF_MODULES_CACHE=/mount/studenten-temp1/users/linku/AL_QA/cache
env: HF_DATASETS_CACHE=/mount/studenten-temp1/users/linku/AL_QA/cache


### arguments.py

args_input.

In [4]:
args_input_ALstrategy = 'RandomSampling'
args_input_initseed = 100 # 1000
args_input_quota = 100 # 1000
args_input_batch = 35 # 128
args_input_dataset_name = 'SQuAD'
args_input_iteration = 3

### load dataset

In [5]:
squad = load_dataset("squad")
# squad["train"] = squad["train"].shuffle(42).select(range(2000))
squad["train"] = squad["train"].select(range(10000))
squad["validation"] = squad["validation"].select(range(5000))

Found cached dataset squad (/home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 98.64it/s]


Next we will preprocess the dataset (training and evaluation data).

In [6]:
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [7]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [8]:
max_length = 384
stride = 128

# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") ### done
# preprocess data
train_dataset = squad["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad["train"].column_names,
)
val_dataset = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)

Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-eb4de106e488b6a5.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-1b312b9152f873ba.arrow


## Evaluation

In [9]:
metric = evaluate.load("squad")

In [10]:
def compute_metrics(start_logits, end_logits, features, examples):
    
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [11]:
class Logger(object):
	def __init__(self, filename="Default.log"):
		self.terminal = sys.stdout
		self.log = open(filename, "a")

	def write(self, message):
		self.terminal.write(message)
		self.log.write(message)

	def flush(self):
		pass

In [12]:
def get_aubc(quota, bsize, resseq):
	# it is equal to use np.trapz for calculation
	ressum = 0.0
	if quota % bsize == 0:
		for i in range(len(resseq)-1):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2

	else:
		for i in range(len(resseq)-2):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2
		k = quota % bsize
		ressum = ressum + ((resseq[-1] + resseq[-2]) * k / 2)
	ressum = round(ressum / quota,3)
	
	return ressum

In [13]:
def get_mean_stddev(datax):
	return round(np.mean(datax),4),round(np.std(datax),4)

# main.py

### parameters

In [14]:
NUM_QUERY = args_input_batch
NUM_INIT_LB = args_input_initseed
NUM_ROUND = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

### seed and device

In [15]:
SEED = 4666
os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### recording

In [16]:
sys.stdout = Logger(os.path.abspath('') + '/logfile/' + DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(1000) + '_normal_log.txt')
warnings.filterwarnings('ignore')

# start experiment
iteration = args_input_iteration

all_acc = []
acq_time = []

In [20]:
# repeate # iteration trials
while (iteration > 0): 
	iteration = iteration - 1

	## data, network, strategy
	net = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)

	## set up training
	training_args = TrainingArguments(
		output_dir="./models",
		evaluation_strategy="no",
		eval_steps=100,
		logging_steps=100,
		learning_rate=1e-4,
		per_device_train_batch_size=8,
		per_device_eval_batch_size=8, 
		gradient_accumulation_steps=1,
		num_train_epochs=3,  # max_steps will override this value
		# max_steps=1000,  # comment out if this is not wanted
		weight_decay=0.01,
		report_to="none",
	)

	## data collator for batching
	data_collator = DefaultDataCollator()

	start = datetime.datetime.now()

	## generate initial labeled pool
	n_pool = len(train_dataset)
	labeled_idxs = np.zeros(n_pool, dtype=bool)

	tmp_idxs = np.arange(n_pool)
	np.random.shuffle(tmp_idxs)
	labeled_idxs[tmp_idxs[:args_input_initseed]] = True

	run_0_labeled_idxs = np.arange(n_pool)[labeled_idxs] 
	
	## record acc performance 
	acc = np.zeros(NUM_ROUND + 1) # build 3 runs + run_0 # origin 10 runs + run_0

	## the actual trainer which performs training and evaluation
	trainer_0 = Trainer(
						model=net,
						args=training_args,
						train_dataset=train_dataset.select(indices=run_0_labeled_idxs),
						eval_dataset=val_dataset,
						tokenizer=tokenizer,
						data_collator=data_collator
						)	
		
	## print info
	print(DATA_NAME)
	# print('RANDOM SEED {}'.format(SEED))
	print(STRATEGY_NAME) # print(type(strategy).__name__)
	
	## round 0 accuracy
	
	trainer_0.train()

	# preds, _, _ = trainer_0.predict(val_dataset)
	# start_logits, end_logits = preds
	# acc[0] = compute_metrics(trainer_0, val_dataset, squad["validation"])['exact_match']

	# print('Round 0\ntesting accuracy {}'.format(acc[0]))
	# print('\n')
	
	# ## round 1 to rd
	# for rd in range(1, NUM_ROUND+1):
	# 	print('Round {}'.format(rd))
		
	# 	## RandomSampling query
	# 	q_idxs = np.random.choice(np.where(labeled_idxs==0)[0], NUM_QUERY, replace=False)
	
	# 	## update
	# 	labeled_idxs[q_idxs] = True
	# 	run_rd_labeled_idxs = np.arange(n_pool)[labeled_idxs]
	# 	print('run_rd_labeled_idxs', len(run_rd_labeled_idxs))

	# 	trainer_rd = Trainer(
	# 						model=net,
	# 						args=training_args,
	# 						train_dataset=train_dataset.select(indices=run_rd_labeled_idxs),
	# 						eval_dataset=val_dataset,
	# 						tokenizer=tokenizer,
	# 						data_collator=data_collator
	# 						)

	# 	## train
	# 	trainer_rd.train()
	
	# 	## round rd accuracy
	# 	acc[rd] = compute_metrics(trainer_rd, val_dataset, squad["validation"])['exact_match']
	# 	print('testing accuracy {}'.format(acc[rd]))
	# 	print('\n')

	# 	torch.cuda.empty_cache()
	
	# ## print results
	# print('SEED {}'.format(SEED))
	# print(STRATEGY_NAME)
	# print(acc)
	# all_acc.append(acc)
	
	# ## save model
	# timestamp = re.sub('\.[0-9]*','_',str(datetime.datetime.now())).replace(" ", "_").replace("-", "").replace(":","")
	# model_path = './modelpara/' + timestamp + DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input_quota)  +'.params'
	# end = datetime.datetime.now()
	# acq_time.append(round(float((end - start).seconds), 3))
	# torch.save(net.state_dict(), model_path)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1608481837.py, line 15)

In [18]:
preds

(array([[-1.656662  , -0.6295358 ,  0.3957903 , ..., -4.165654  ,
         -4.16063   , -4.1774597 ],
        [-1.6779928 , -0.63508624,  0.4033468 , ..., -4.1644654 ,
         -4.158975  , -4.1757402 ],
        [-1.7496563 , -0.7205538 , -1.0128822 , ..., -4.1720414 ,
         -4.174626  , -4.1854897 ],
        ...,
        [-1.5591328 , -0.8819545 , -1.4846373 , ..., -4.152286  ,
         -4.2612066 , -4.1691113 ],
        [-2.0682573 , -0.8054493 , -0.7898907 , ..., -4.1615424 ,
         -4.235041  , -4.1808558 ],
        [-2.5107467 , -0.98651755, -0.9962488 , ..., -4.0194745 ,
         -4.0293827 , -4.100231  ]], dtype=float32),
 array([[-0.67598474, -0.81034905,  0.8880169 , ..., -4.2177196 ,
         -4.211968  , -4.216345  ],
        [-0.6761083 , -0.82706827,  0.8931461 , ..., -4.220371  ,
         -4.2148547 , -4.2194457 ],
        [-0.77423054, -1.0974021 , -1.0347588 , ..., -4.1096992 ,
         -4.0902014 , -4.07936   ],
        ...,
        [-0.02477671, -1.2988623 , -1.5

In [1]:
acc

NameError: name 'acc' is not defined

In [18]:
all_acc

[array([5.96, 5.96, 5.96])]

In [20]:
# cal mean & standard deviation
acc_m = []
file_name_res_tot = DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input_quota) + '_normal_res_tot.txt'
file_res_tot =  open(os.path.join(os.path.abspath('') + '/results', '%s' % file_name_res_tot),'w')

file_res_tot.writelines('dataset: {}'.format(DATA_NAME) + '\n')
file_res_tot.writelines('AL strategy: {}'.format(STRATEGY_NAME) + '\n')
file_res_tot.writelines('number of labeled pool: {}'.format(NUM_INIT_LB) + '\n')
file_res_tot.writelines('number of unlabeled pool: {}'.format(len(train_dataset) - NUM_INIT_LB) + '\n')
file_res_tot.writelines('number of testing pool: {}'.format(len(val_dataset)) + '\n')
file_res_tot.writelines('batch size: {}'.format(NUM_QUERY) + '\n')
file_res_tot.writelines('quota: {}'.format(NUM_ROUND*NUM_QUERY)+ '\n')
file_res_tot.writelines('time of repeat experiments: {}'.format(args_input_iteration)+ '\n')

# result
for i in range(len(all_acc)):
	acc_m.append(get_aubc(args_input_quota, NUM_QUERY, all_acc[i]))
	print(str(i)+': '+str(acc_m[i]))
	file_res_tot.writelines(str(i)+': '+str(acc_m[i])+'\n')
mean_acc, stddev_acc = get_mean_stddev(acc_m)
mean_time, stddev_time = get_mean_stddev(acq_time)

print('mean AUBC(acc): '+str(mean_acc)+'. std dev AUBC(acc): '+str(stddev_acc))
print('mean time: '+str(mean_time)+'. std dev time: '+str(stddev_time))

file_res_tot.writelines('mean acc: '+str(mean_acc)+'. std dev acc: '+str(stddev_acc)+'\n')
file_res_tot.writelines('mean time: '+str(mean_time)+'. std dev acc: '+str(stddev_time)+'\n')

# save result

file_name_res = DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input_quota) + '_normal_res.txt'
file_res =  open(os.path.join(os.path.abspath('') + '/results', '%s' % file_name_res),'w')


file_res.writelines('dataset: {}'.format(DATA_NAME) + '\n')
file_res.writelines('AL strategy: {}'.format(STRATEGY_NAME) + '\n')
file_res.writelines('number of labeled pool: {}'.format(NUM_INIT_LB) + '\n')
# file_res.writelines('number of unlabeled pool: {}'.format(dataset.n_pool - NUM_INIT_LB) + '\n')
file_res.writelines('number of unlabeled pool: {}'.format(len(train_dataset) - NUM_INIT_LB) + '\n')
# file_res.writelines('number of testing pool: {}'.format(dataset.n_test) + '\n')
file_res.writelines('number of testing pool: {}'.format(len(val_dataset)) + '\n')
file_res.writelines('batch size: {}'.format(NUM_QUERY) + '\n')
file_res.writelines('quota: {}'.format(NUM_ROUND*NUM_QUERY)+ '\n')
file_res.writelines('time of repeat experiments: {}'.format(args_input_iteration)+ '\n')
avg_acc = np.mean(np.array(all_acc),axis=0)
for i in range(len(avg_acc)):
	tmp = 'Size of training set is ' + str(NUM_INIT_LB + i*NUM_QUERY) + ', ' + 'accuracy is ' + str(round(avg_acc[i],4)) + '.' + '\n'
	file_res.writelines(tmp)

file_res.close()
file_res_tot.close()

In [19]:
raw_predictions = trainer_0.predict(val_dataset)

NameError: name 'trainer_rd' is not defined

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers